In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [4]:
original = pd.read_csv("../../data/processed/original (2).csv")
campaign = pd.read_csv("../../data/campaign_data.csv", sep = ';')
# https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets?select=test.csv
# training data was used since it was much larger than test data (randomly selected rows from training data)

In [6]:
# similar columns: education, marital, age
common_features = ['education', 'marital', 'age']

ori = original.copy()
ori['education'] = ori['Education_Level']
ori['education'] = ori['education'].replace(
    {'Doctorate': 'tertiary',
     'Post-Graduate': 'tertiary',
     'Graduate': 'tertiary',
     'College': 'secondary',
     'High School': 'secondary', 
     'Uneducated': 'primary'})

ori['marital'] = ori['Marital_Status']
ori['marital'] = ori['marital'].replace(
    {'Married': 'married',
     'Single': 'single',
     'Divorced': 'divorced'})

ori['age'] = ori['Customer_Age']

Data that will be added:

- `duration`: last contact duration, in seconds (numeric) --> `Duration_of_Contact`
- `campaign`: number of contacts performed during this campaign and for this client (numeric, includes last contact) --> `Number_of_Contacts_Made`
- `pdays`: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted) --> `Last_Contact_Made`
- `y`: has the client subscribed a term deposit? (binary: "yes","no") --> `Outcome`

In [9]:
# d/b of duration
duration_db = (
    campaign.groupby(common_features)['duration']
    .value_counts(normalize = True)
    .unstack(fill_value = 0)
)

# d/b of campaign
campaign_db = (
    campaign.groupby(common_features)['campaign']
    .value_counts(normalize = True)
    .unstack(fill_value = 0)
)

# d/b of pdays
pdays_db = (
    campaign.groupby(common_features)['pdays']
    .value_counts(normalize = True)
    .unstack(fill_value = 0)
)

# d/b of y
campaign['y'] = campaign['y'].replace({'yes': 1, 'no': 0})
y_db = (
    campaign.groupby(common_features)['y']
    .mean()
    .reset_index()
)

In [11]:
# generating synthetic features
def generate_synthetic_features(df, distribution, feature):
    synthetic_feature = []
    
    for _, row in df.iterrows():
        features = tuple(row[common_features])
        try:
            sample = np.random.choice(
                distribution.columns,
                p=distribution.loc[features].values
            )
        except KeyError: #incase the feature combination is missing
            sample = np.random.choice(distribution.columns)
            
        synthetic_feature.append(sample)
            
    return synthetic_feature

def generate_synthetic_outcome(df, distribution):
    synthetic_outcome = []
    
    for _, row in df.iterrows():
        features = tuple(row[common_features])
        matching_row = distribution.loc[(distribution[common_features] == features).all(axis=1)]
        
        if not matching_row.empty:
            prob_yes = matching_row['y'].values[0]
            sample = np.random.choice([0, 1], p=[1 - prob_yes, prob_yes])
        else:
            sample = np.random.choice([0, 1])
        
        synthetic_outcome.append(sample)
    
    return synthetic_outcome

# generating the data
np.random.seed(10)
original['Duration_of_Contact'] = generate_synthetic_features(ori, duration_db, 'duration')
original['Num_of_Contacts_Made'] = generate_synthetic_features(ori, campaign_db, 'campaign')
original['Last_Contacted'] = generate_synthetic_features(ori, pdays_db, 'pdays')
original['Outcome'] = generate_synthetic_outcome(ori, y_db)
original.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Savings,Duration_of_Contact,Num_of_Contacts_Made,Last_Contacted,Outcome
0,768805383,Existing Customer,45,M,3,High School,Married,60 - 80,Blue,39,...,1.335,1144,42,1.625,0.061,877105.94,304,1,-1,0
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than 40,Blue,44,...,1.541,1291,33,3.714,0.105,3788.54,35,1,-1,0
2,713982108,Existing Customer,51,M,3,Graduate,Married,80 - 120,Blue,36,...,2.594,1887,20,2.333,0.000,782871.18,209,2,-1,0
3,709106358,Existing Customer,40,M,3,Uneducated,Married,60 - 80,Blue,21,...,2.175,816,28,2.500,0.000,455824.95,327,1,-1,1
4,713061558,Existing Customer,44,M,2,Graduate,Married,40 - 60,Blue,36,...,1.376,1088,24,0.846,0.311,564743.40,157,7,-1,0


In [14]:
original.to_csv('../../data/processed/original (3).csv', index=False)